## 문제 1

### 1-1. 분류 평가지표 중 '정밀도'와 '재현율'에 대한 개념과 이 둘은 어떤 관계인지 설명해주세요.

정밀도란 모델이 positive라고 분류한 것 중에서 실제 True인 것을 비율을 의미합니다.

재현율이란 실제 값이 True인 것 중에서 모델이 positive라고 예측한 것의 비율입니다.

상충하는 관계(trade-off)입니다. 정밀도가 향상되면 재현율이 감소합니다.



***

## 문제 2

### 정밀도와 재현율이 각각 더 중요한 사례를 들어주세요.
- 정밀도가 중요한 사례와 재현율이 중요한 사례 총 2가지를 적어주셔야 합니다.
- 되도록이면 PPT 이외의 사례를 들어주세요!

재현율 : atm기계에 돈을 넣을 경우 실제 값이 10000원인데 10000원이라고 알맞게 예측을 해야 하기 때문에 재현율이 중요한것 같습니다.

정밀도 : 기상을 예측한 결과 내일 비가 온다고 하였는데 실제로 비가 온 경우를 판단하는 개념으로 일기예보에서도 정밀도가 중요한것 같습니다. 

***

## 문제3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import Binarizer

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

### [ 종양종류 예측해보기 ]
- load_breast_cancer은 위스콘신 유방암 데이터로 악성종양(malignant)과 양성종양(benign)으로 구분되는 데이터세트 입니다.
- 타깃 레이블 값의 종류는 악성이 '0'으로 양성이 '1'로 되어 있습니다.

#### 기본 제공 코드

In [ ]:
# 데이텃셋 로드 및 X,y 지정
dataset = load_breast_cancer()
X_features=dataset.data
y_label=dataset.target

In [ ]:
# `train_test_split`으로 train, test 데이터 분리

X_train,X_test,y_train,y_test = train_test_split(X_features, y_label, test_size=0.3, random_state=11)
print(X_train.shape, X_test.shape)

(398, 30) (171, 30)


In [ ]:
# RandomForest 객체 생성 후 예측 수행

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
pred_proba = rf_clf.predict_proba(X_test)[:,1]

### 3-1. 정확도, 정밀도, 재현율, 오차행렬을 산출하세요.
- get_clf_eval 함수를 활용하세요.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test,pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차행렬')
  print(confusion)
  print('정확도: {0:.4f}, 정밀도: {1: .4f}, 재현율: {2: .4f}'.format(accuracy, precision, recall))

In [ ]:
get_clf_eval(y_test, pred)

오차행렬
[[ 56   5]
 [  1 109]]
정확도: 0.9649, 정밀도:  0.9561, 재현율:  0.9909


### 3-2. 임계값이 0.3, 0.5, 0.7, 0.9일때 오차행렬, 정확도, 정밀도, 재현율을 구하세요.

In [ ]:
from sklearn.preprocessing import Binarizer

In [ ]:
thresholds = [ 0.3, 0.5, 0.7, 0.9]

In [ ]:
pred_proba = rf_clf.predict_proba(X_test)
pred_proba_1 = pred_proba[: ,1].reshape(-1,1)
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
  for custom_threshold in thresholds:
    binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
    custom_predict = binarizer.transform(pred_proba_c1)
    print('임계값 : ', custom_threshold)
    get_clf_eval(y_test, custom_predict)
    print('')

get_eval_by_threshold(y_test,pred_proba_1, thresholds)

임계값 :  0.3
오차행렬
[[ 51  10]
 [  0 110]]
정확도: 0.9415, 정밀도:  0.9167, 재현율:  1.0000

임계값 :  0.5
오차행렬
[[ 56   5]
 [  1 109]]
정확도: 0.9649, 정밀도:  0.9561, 재현율:  0.9909

임계값 :  0.7
오차행렬
[[ 59   2]
 [  4 106]]
정확도: 0.9649, 정밀도:  0.9815, 재현율:  0.9636

임계값 :  0.9
오차행렬
[[59  2]
 [11 99]]
정확도: 0.9240, 정밀도:  0.9802, 재현율:  0.9000



### 3-3. 위의 임곗값들 중 어떤 것이 가장 적절할까요? 답과 그 이유를 적어주세요.

현재 데이터는 양성종양, 악성종양을 구분하여야 합니다. 정확도도 중요하지만 재현율이 높아야 합니다. 실제 악성종양인 사람을 양성종양으로 분류하여서는 안되기 때문에 임계값이 0.3이 가장 적절합니다.

***

## 문제 4

### 4-1. F1 스코어의 개념과 어떨 때 상대적으로 높은 값를 가지는지 적어주세요.

F1 score은 재현율과 정밀도의 조화평균입니다.

재현율과 정밀도 어느 한쪽으로 치우치지 않는 수치를 나타내는 경우 상대적으로 높은 값을 가지는것 같습니다.

### 4-2. F1 스코어를 구하세요.

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, pred)
print(f'fl score: {f1: .3f}')

fl score:  0.973


### 4-3. `get_clf_eval()` 함수에 F1 스코어를 구하는 코드를 추가하세요.

In [ ]:
def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  f1 = f1_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f}, F1:{3: .4f}'.format(accuracy, precision, recall, f1))
get_clf_eval(y_test,pred)

오차 행렬
[[ 56   5]
 [  1 109]]
정확도:  0.9649, 정밀도:  0.9561, 재현율:  0.9909, F1: 0.9732


### 4-4. <U>F1 스코어 >= 0.98</U>이 되도록 값을 출력하세요.  (다른 평가 지표의 값은 무시)
  
`Hint : 임계값 조정`

In [ ]:
thresholds = [0.6]
pred_proba = rf_clf.predict_proba(X_test)
pred_proba_1 = pred_proba[: ,1].reshape(-1,1)
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
  for custom_threshold in thresholds:
    binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
    custom_predict = binarizer.transform(pred_proba_c1)
    print('임계값 : ', custom_threshold)
    get_clf_eval(y_test, custom_predict)
    print('')

get_eval_by_threshold(y_test,pred_proba_1, thresholds)

임계값 :  0.6
오차 행렬
[[ 59   2]
 [  2 108]]
정확도:  0.9766, 정밀도:  0.9818, 재현율:  0.9818, F1: 0.9818



***

## 문제 5

### 5-1. ROC-AUC가 무엇인지 설명하세요. 

In [ ]:
# ROC 곡선 : 거짓 양성 비율에 대한 진짜 양성 비율의 곡선입니다.
# TPR(True Positive Rate) :실제 양성을 제데로 양성이라고 예측한 경우입니다. 
# TNR(True Negative Rate) :실제 음성을 제대로 음성이라고 예측한 경우입니다.   ->TPR과 서로 tradeoff 관계입니다.
# AUC(Area Under Curve) : ROC곡선의 면적의 값입니다.

### 5-2. ROC-AUC 값을 구하세요. 

In [ ]:
from sklearn.metrics import roc_auc_score

rf_pred_proba = rf_clf.predict_proba(X_test)[:,1]
rf_roc_score = roc_auc_score(y_test, rf_pred_proba)
print('rf_ROC AUC 값 : {0: .4f}'.format(rf_roc_score))


rf_ROC AUC 값 :  0.9903
